In [1]:
# 进入mmsegmentation目录
import os
import workspace_path
os.chdir(workspace_path.path)

## 视频预测-Python API
不推荐命令行模式，推荐python API模式，比命令行模式快

In [2]:
# 导包
from mmseg.utils import register_all_modules
register_all_modules()

### 加载模型

In [3]:
# 模型 config 配置文件
config_file = 'configs/segformer/segformer_mit-b5_8xb1-160k_cityscapes-1024x1024.py'
# 模型 checkpoint 权重文件
checkpoint_file = 'myFolder/checkpoint/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

from mmseg.apis import init_model
model = init_model(config_file, checkpoint_file, device='cuda:0')

from mmengine.model.utils import revert_sync_batchnorm
import torch
if not torch.cuda.is_available():
    model = revert_sync_batchnorm(model)

E:\Code\openmmlab\mmsegmentation\mmseg\models\builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
E:\Code\openmmlab\mmsegmentation\mmseg\models\losses\cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: myFolder/checkpoint/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth


## 输入视频

In [4]:
input_video = 'data/street_20220330_174028.mp4'
# 创建临时文件夹存放单帧结果
import time
temp_out_dir = time.strftime('%Y%m%d%H%M%S')
os.mkdir(temp_out_dir)
print('创建临时文件夹 {} 用于存放每帧预测结果'.format(temp_out_dir))

创建临时文件夹 20230807170131 用于存放每帧预测结果


## 视频单帧预测

In [5]:
# 获取 Cityscapes 街景数据集 类别名和调色板
from mmseg.datasets import cityscapes
classes = cityscapes.CityscapesDataset.METAINFO['classes']
palette = cityscapes.CityscapesDataset.METAINFO['palette']

from mmseg.apis import inference_model
import numpy as np
from PIL import Image
def predict_single_frame(img, opacity=0.2):
    result = inference_model(model, img)
    
    # 将分割图按调色板染色
    seg_map = np.array(result.pred_sem_seg.data[0].detach().cpu().numpy()).astype('uint8')
    seg_img = Image.fromarray(seg_map).convert('P')
    seg_img.putpalette(np.array(palette, dtype=np.uint8))
    show_img = (np.array(seg_img.convert('RGB')))*(1-opacity) + img*opacity
    return show_img

## 视频逐帧预测

In [6]:
# 读入待预测视频
import mmcv
video = mmcv.VideoReader(input_video)
import mmengine
prog_bar = mmengine.ProgressBar(len(video))

# 对视频逐帧处理
import cv2
for frame_id, img in enumerate(video):
    show_img = predict_single_frame(img, opacity=0.15)
    temp_path = f'{temp_out_dir}/{frame_id:06d}.jpg'
    # 保存语义分割预测结果图像至临时文件夹
    cv2.imwrite(temp_path, show_img)
    # 更新进度条
    prog_bar.update()

# 把每一帧串成视频文件
mmcv.frames2video(temp_out_dir, 'myFolder/outputs/B3_video.mp4', fps=video.fps, fourcc='mp4v')
print('删除临时文件夹', temp_out_dir)
import shutil
shutil.rmtree(temp_out_dir)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 138/138, 34.0 task/s, elapsed: 4s, ETA:     0s[                                                  ] 0/138, elapsed: 0s, ETA:
删除临时文件夹 20230807170131
